# MNIST hand written digits recognition with deep learning 
---
## projet réalisé par : ELARNI Maroua & SADKI Ayoub
---



In [393]:
import os
import numpy as np 
import scipy as sc 
import pandas as pd
from scipy.io import loadmat
from matplotlib import pyplot as plt


In [394]:
 #to normalize input data
    def normalize(x):
    x = x/255.
    return x

#k = 21
#to subfold data 
def define_datafolds(k, S , y):
    data_set_folds =[]
    labels_folds=[]
    v= y.shape
    for i in range(k):
        data_set_folds.append( S[int(y.shape[0]/k)*i : int(y.shape[0]/k)*(i+1),:] )   
        labels_folds.append( y[int(y.shape[0]/k)*i : int(y.shape[0]/k)*(i+1)] )   
    return np.array(data_set_folds) , np.array(labels_folds)

In [395]:
#input , layer n nodes , output
#to create the flexible neurone network 
def make_nn( input_dim = 784 , nLayers = 18 , nNodes = 17 , out_dim= 10 ) :
    weights_matrices = []
    biais_matrices = []
    nHiddenLayers = nLayers - 2
    
    input_weights = weights_matrices.append(   np.random.rand(nNodes, input_dim) -0.5)
    biais_input = biais_matrices.append(   np.random.rand(nNodes, 1) -0.5)

#     input_weights = weights_matrices.append(   np.ones((nNodes, input_dim)))
#     biais_input = biais_matrices.append(   np.ones((nNodes, 1)))

    for i in range(nHiddenLayers-1):
        weights_matrices.append(   np.random.rand(nNodes, nNodes)-0.5)
        biais_matrices.append(   np.random.rand(nNodes, 1)-0.5)

#         weights_matrices.append(np.ones((nNodes, nNodes)))
#         biais_matrices.append( np.ones((nNodes, 1)))
    
    output_weights = weights_matrices.append(   np.random.rand(out_dim, nNodes)-0.5)
    biais_output = biais_matrices.append(   np.random.rand(out_dim, 1)-0.5)

#     output_weights = weights_matrices.append(   np.ones((out_dim, nNodes)))
#     biais_output = biais_matrices.append(   np.ones((out_dim, 1)))
    
    return weights_matrices , biais_matrices
    
    

In [526]:
def Relu(x):
    return np.maximum(0,x)

def ReLU_deriv(Z):
    return Z > 0

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def forward_propagation(weights_matrices , biais_matrices, x):
    lin_out = []
    act_out = []
    print("input shape :" , x.shape)
    #x = x.reshape((x.shape[0],400))
    x = x.T
    for i in range(len(weights_matrices)-1):
       # print("loop: ", i)
       # print("weights shape" , np.array(weights_matrices[i]).shape)
        #print("biais shape" , np.array(biais_matrices[i]).shape)
        x = (np.array(weights_matrices[i]) @ x) + np.array(biais_matrices[i])
        lin_out.append(x)
        x = Relu(x)
        act_out.append(x)
        #print("New input shape:", x.shape)
    y_pred = softmax(np.array(weights_matrices[-1]) @ x + np.array(biais_matrices[-1]))
    return lin_out, act_out ,y_pred

def one_hot_encode(y):
    one_hot = np.zeros((10 , y.shape[0])).T
    for i in range(y.shape[0]):
            one_hot[i][y[i]] = 1  
    return one_hot.T

def backward_propagation(lin_out, act_out, y_pred, weights_matrices, y,  x):
    m = x.shape[0]
    y_enc = one_hot_encode(y)
    #print("y_enc", y_enc.shape)
    #print("y_pred", y_pred.shape)
    dZ, dW, dB =[None]*len(weights_matrices) , [None]*len(weights_matrices), [None]*len(weights_matrices)
    #print("dZ=",dZ.shape)
    dZ[0]= - y_enc + y_pred 
    #print("dZ[0]=",dZ[0].shape)
    #print("actout[0]=",act_out[-1].shape)
    dW[0] = (1/m)*dZ[0].dot(act_out[-1].T)# dW(4)
    dB[0] = (1/m)*np.sum(dZ[0] , axis = 1)
    #print("dz0:  " , dZ[0])
    #print("db0 = " , dB[0])
    for i in range(len(weights_matrices)-1,1,-1):
       # print(i)
      # print(len(lin_out))
        dZ[len(weights_matrices)-i] = weights_matrices[i].T.dot(dZ[len(weights_matrices)-i-1]) * ReLU_deriv(lin_out[i-1])
        dW[len(weights_matrices)-i] = (1/m) * dZ[len(weights_matrices)-i].dot(act_out[i-2].T)
        dB[len(weights_matrices)-i]  = (1/m) * np.sum(dZ[len(weights_matrices)-i], axis = 1)
    dZ[len(weights_matrices)-1] = weights_matrices[1].T.dot(dZ[len(weights_matrices)-2]) * ReLU_deriv(lin_out[0])
    dW[len(weights_matrices)-1] = (1/m) * dZ[len(weights_matrices)-1].dot(x)
    dB[len(weights_matrices)-1]  =( 1/m) * np.sum(dZ[len(weights_matrices)-1] , axis = 1)
    #print(len(dW), len(dB))
    return dW , dB
#     return np.array(dW) ,np.array(dB)

def update_params(weights_matrices, dW, biais_matrices, dB, alpha):
    #print("dw shape : " , dW[16].shape)
#     print("weig mat shape : " , weights_matrices[0].shape)
    dW=dW[::-1]
    dB=dB[::-1]
#     print("dw :" , dW)
#     print("we mat : " ,weights_matrices)
    #print("db : ", dB)
    #print("biais : \n" , biais_matrices)
    for i in range(len(weights_matrices)) :
        weights_matrices[i] = weights_matrices[i] - alpha * dW[i]
        #print( "biais before: " , biais_matrices[i] , " shape : " , biais_matrices[i].shape)
        #print( "dB before: " , dB[i] , " shape : " , dB[i].shape)
        biais_matrices[i] = biais_matrices[i] - alpha * dB[i].reshape((dB[i].shape[0] , 1))
#         print("biais after \n" , biais_matrices )
    return weights_matrices, biais_matrices

def gradient_descent(X,Y, alpha, iterations):
    weights_matrices , biais_matrices = make_nn( input_dim = 784 , nLayers = 3 , nNodes = 4, out_dim= 10)
    for i in range(iterations):
        print("Iteration: ", i)
        print(weights_matrices[0])
        lin_out, act_out ,y_pred = forward_propagation(weights_matrices , biais_matrices, X)
        dW, dB = backward_propagation(lin_out, act_out ,y_pred, weights_matrices, Y, X)
        weights_matrices, biais_matrices = update_params(weights_matrices, dW, biais_matrices, dB, alpha)
    _,_,y_pred = forward_propagation(weights_matrices , biais_matrices, X)
    return weights_matrices, biais_matrices, y_pred 

def get_accuracy(predictions, Y):
    return str(round((np.sum(np.argmax(predictions, axis=0) == Y) / Y.shape)[0],2)*100) + '%'

def predict(weights_matrices, biais_matrices, Xtest):
    _,_,y_test = forward_propagation(weights_matrices , biais_matrices, Xtest)
    return y_test
    

### les données utilisés ne sont pas celles du tp 4:
---
 on a rencontré trop de problème en les utilisant et pour contourner ce problème on a eu recourt a une autre base de donnée MNIST qu'on a récupérer sur kaggle (train.csv) , on a travaillé avec 10000 images pour le training et 1000 pour le test


In [450]:
data = pd.read_csv('Data/train.csv')

In [451]:
#spliting data to train and test :
X_train = data.to_numpy()[:10000 , 1:]
y_train = data.to_numpy()[:10000 , 0]

X_test = data.to_numpy()[10000:11000 , 1:]
y_test = data.to_numpy()[10000:11000 , 0]

#normalisation des données :
X = normalize(X_train)

# Training :

In [ ]:
nbr_iter = 3000
alpha = 0.1
weights_matrices, biais_matrices, y_pred = gradient_descent(X , y , alpha, nbr_iter)

In [400]:
# data_set_folds , labels_folds = define_datafolds(21, X , y)
# X_train = np.delete(data_set_folds, 13 , axis=0)
# y_train =  np.delete(labels_folds, 13, axis=0)
# X_test = data_set_folds[13]
# y_test = labels_folds[13]

# Execution on test Set:

In [504]:
X_test = normalize(X_test)
f = predict(weights_matrices, biais_matrices, X_test)

# Performance measures on a test data:


In [529]:
accuracy = get_accuracy(y_pred , y)
print("Accuracy:", accuracy)
from sklearn.metrics import recall_score, precision_score
precision = round(precision_score(y_test,np.argmax(f, axis=0),  average='macro'),2)
print("Precision: ",str(precision*100),"%")
recall = round(recall_score(y_test,np.argmax(f, axis=0),  average='macro'),2)
print("Recall:",str(recall*100),"%")

Accuracy: 86.0%
Precision:  84.0 %
Recall: 84.0 %
